In [10]:
from keras.layers.core import Activation, Dense
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import nltk  #用来分词
import collections  #用来统计词频
import numpy as np

D:\study\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 预处理
获取每条评论的最大长度, 以及所有评论中不重复词的个数 

建立词到ID的映射, 保存在 word2ID  ID2word

每条数据的形式如下:

1	The Da Vinci Code book is just awesome.

In [26]:
'1 The Da Vinci Code book is just awesome.'.strip().split(' ')

['1', 'The', 'Da', 'Vinci', 'Code', 'book', 'is', 'just', 'awesome.']

In [23]:
with open(r'D:\CS\dataset\NLP\英文情感分类数据集(二分类).txt', 'r+', encoding='utf-8') as f:
    for line in f:
        label, comment = line.strip().split('\t')

1	The Da Vinci Code book is just awesome.

1	this was the first clive cussler i've ever read, but even books like Relic, and Da Vinci code were more plausible than this.

1	i liked the Da Vinci Code a lot.

1	i liked the Da Vinci Code a lot.

1	I liked the Da Vinci Code but it ultimatly didn't seem to hold it's own.

1	that's not even an exaggeration ) and at midnight we went to Wal-Mart to buy the Da Vinci Code, which is amazing of course.

1	I loved the Da Vinci Code, but now I want something better and different!..

1	i thought da vinci code was great, same with kite runner.

1	The Da Vinci Code is actually a good movie...

1	I thought the Da Vinci Code was a pretty good book.

1	The Da Vinci Code is one of the most beautiful movies ive ever seen.

1	The Da Vinci Code is an * amazing * book, do not get me wrong.

1	then I turn on the light and the radio and enjoy my Da Vinci Code.

1	The Da Vinci Code was REALLY good.

1	i love da vinci code....

1	i loved da vinci code..

1	TO NIGH


1	Mission Impossible 3 was excellent.

1	So as felicia's mom is cleaning the table, felicia grabs my keys and we dash out like freakin mission impossible.

1	I liked the first " Mission Impossible.

1	the last stand and Mission Impossible 3 both were awesome movies.

1	i love kirsten / leah / kate escapades and mission impossible tom as well...

1	mission impossible 2 rocks!!....

1	we're gonna like watch Mission Impossible or Hoot.(

1	Which is why i said silent hill turned into reality coz i was hella like goin mission impossible down that bitch.

1	i love being a sentry for mission impossible and a station for bonkers.

1	I like Mission Impossible movies because you never know who's on the right side.

1	Mission Impossible 3 was excellent.

1	So as felicia's mom is cleaning the table, felicia grabs my keys and we dash out like freakin mission impossible.

1	I liked the first " Mission Impossible.

1	the last stand and Mission Impossible 3 both were awesome movies.

1	i love kirsten

1	man i loved brokeback mountain!

1	dudeee i LOVED brokeback mountain!!!!

1	I either LOVE Brokeback Mountain or think it's great that homosexuality is becoming more acceptable!:

1	Anyway, thats why I love " Brokeback Mountain.

1	Brokeback mountain was beautiful...

1	I love Brokeback Mountain.

1	I love Brokeback Mountain....

1	He's like,'YEAH I GOT ACNE AND I LOVE BROKEBACK MOUNTAIN '..

1	Brokeback Mountain was so awesome.

1	Brokeback Mountain was an AWESOME movie.

1	man i loved brokeback mountain!

1	dudeee i LOVED brokeback mountain!!!!

1	I either LOVE Brokeback Mountain or think it's great that homosexuality is becoming more acceptable!:

1	Anyway, thats why I love " Brokeback Mountain.

1	Brokeback mountain was beautiful...

1	I love Brokeback Mountain.

1	I love Brokeback Mountain....

1	He's like,'YEAH I GOT ACNE AND I LOVE BROKEBACK MOUNTAIN '..

1	Brokeback Mountain was so awesome.

1	Brokeback Mountain was an AWESOME movie.

1	man i loved brokeback mountain!

1	dudee


0	by the way, the Da Vinci Code sucked, just letting you know...

0	i heard da vinci code sucked soo much only 2.5 stars:

0	da vinci code sucks...

0	Da Vinci Code sucks.

0	friday hung out with kelsie and we went and saw The Da Vinci Code SUCKED!!!!!

0	Combining the opinion / review from Gary and Gin Zen, The Da Vinci Code sucks.

0	Da Vinci Code sucks be...

0	Da Vinci Code sucked..

0	The Da Vinci Code sucked big time.

0	Da Vinci Code = Up, Up, Down, Down, Left, Right, Left, Right, B, A, SUCK!

0	by the way, the Da Vinci Code sucked, just letting you know...

0	i heard da vinci code sucked soo much only 2.5 stars:

0	da vinci code sucks...

0	Da Vinci Code sucks.

0	friday hung out with kelsie and we went and saw The Da Vinci Code SUCKED!!!!!

0	Combining the opinion / review from Gary and Gin Zen, The Da Vinci Code sucks.

0	Da Vinci Code sucks be...

0	Da Vinci Code sucked..

0	The Da Vinci Code sucked big time.

0	Da Vinci Code = Up, Up, Down, Down, Left, Right, Left, Right, 

0	I did lapse once, created a new account, and left two stupid / short comments on the Brokeback Mountain board.

0	becuase that stupid brokeback mountain movie.

0	Brokeback Mountain was depressing.

0	now I dont know whether I consider Brokeback Mountain to be the shittiest movie ive watched recently.

0	I also disliked Brokeback Mountain.

0	1-BROKEBACK MOUNTAIN IS A STUPID MOVIE.

0	which is probably why I disliked Brokeback Mountain...

0	Stupid brokeback mountain reference i did.

0	I thought Brokeback Mountain was a terrible movie.

0	And I still think " Brokeback Mountain " was a horrible movie.

0	Just for the record, Brokeback Mountain was a terrible movie.

0	We watched Brokeback Mountain, which was pretty horrible.

0	Brokeback Mountain is so depressing.(

0	I thought Brokeback Mountain was incredibly depressing...

0	Which answers why I dislike brokeback mountain...

0	I don't care what anyone says, Brokeback Mountain was a shitty movie...

0	Because, this kid wrote a lett

In [12]:
maxlen = 0  #句子最大长度
word_freqs = collections.Counter()  #词频
num_recs = 0 # 样本数
with open(r'D:\CS\dataset\NLP\英文情感分类数据集(二分类).txt', 'r+') as f:
    for line in f:
        label, sentence = line.strip().split("\t")
        words = nltk.word_tokenize(sentence.lower())
        if len(words) > maxlen:
            maxlen = len(words)
        for word in words:
            word_freqs[word] += 1
        num_recs += 1
print('max_len ',maxlen)
print('nb_words ', len(word_freqs))

UnicodeDecodeError: 'gbk' codec can't decode byte 0xbd in position 2270: illegal multibyte sequence